# Combining data on degradation of curves in Malmbanan.

We start by reading data from all the bandelar 111, 113, 116-119 with filenames in the form of RPM Rälslitage i kurva_111.csv. We then combine them into one dataframe.

In [1]:
import pandas as pd
import glob
import os

# Step 1: Define the file pattern for the specific bandel
file_pattern = "RPM Rälslitage i kurva_111.csv RPM Rälslitage i kurva_113.csv RPM Rälslitage i kurva_116.csv RPM Rälslitage i kurva_117.csv RPM Rälslitage i kurva_118.csv RPM Rälslitage i kurva_119.csv"

# Correct the file pattern to use a proper glob pattern
file_pattern = "RPM Rälslitage i kurva_*.csv"

# Step 2: Read all files matching the pattern
file_list = glob.glob(file_pattern)

# Step 3: Create a list to hold the DataFrames
dataframes = []


# Function to convert comma to dot in numeric columns
def convert_comma_to_dot(df):
    # Iterate through all columns
    for col in df.select_dtypes(include=['object']).columns:
        try:
            # Try to convert column, replacing comma with dot
            df[col] = df[col].str.replace(',', '.', regex=False)
            
            # Try to convert to numeric, coercing errors to NaN
            numeric_col = pd.to_numeric(df[col], errors='coerce')
            
            # If conversion is successful, replace the column
            if not numeric_col.isna().all():
                df[col] = numeric_col
        except Exception as e:
            print(f"Could not convert column {col}: {e}")
    
    return df


# Read files
for file in file_list:
    try:
        # Read CSV with semicolon delimiter
        df = pd.read_csv(file, delimiter=';', encoding='ISO-8859-1')
        
        # Convert comma to dot in numeric columns
        df = convert_comma_to_dot(df)
        
        if not df.empty:
            dataframes.append(df)
            print(f"Successfully loaded {file}")
    except Exception as e:
        print(f"Error reading {file}: {e}")

# Check if any dataframes were loaded
if not dataframes:
    print("No files were successfully loaded. Please check the file pattern and file locations.")
    exit()

Successfully loaded RPM Rälslitage i kurva_111.csv
Successfully loaded RPM Rälslitage i kurva_113.csv
Successfully loaded RPM Rälslitage i kurva_116.csv
Successfully loaded RPM Rälslitage i kurva_117.csv
Successfully loaded RPM Rälslitage i kurva_118.csv
Successfully loaded RPM Rälslitage i kurva_119.csv


Combine all the bandelar in one dataframe.

In [2]:
# Step 5: Combine all DataFrames into one
combined_df = pd.concat(dataframes, ignore_index=True)

We need to process some columns to get correct format of info.

In [3]:
import re

# Function to extract position from location string
def extract_position(value):
    """Extract numeric value from Startläge/Slutläge."""
    if pd.isna(value):
        return None
    
    # Replace comma with dot for consistent parsing
    value = str(value).replace(',', '.')
    
    position = re.search(r'(\d+)(\s*\+\s*)(\d+)', value)
    if position:
        return int(position.group(1))*1000 + int(position.group(3))  # Convert to meters
    return None

# Function to extract measurement date
def extract_measurement_date(value):
    """Extract date and time from Mätning Spårvidd."""
    if pd.isna(value):
        return None
    
    match = re.search(r'SPL_(\d{6})(\d{4})', str(value))
    if match:
        date = match.group(1)  # e.g., 150727
        time = match.group(2)  # e.g., 0844
        formatted_date = f"20{date[:2]}-{date[2:4]}-{date[4:]} {time[:2]}:{time[2:]}"
        return formatted_date
    return None

# Data Processing
def process_dataframe(df):
    # Create an explicit copy to avoid SettingWithCopyWarning
    processed_df = df.copy()
    
    # Remove rows with missing gauge widening measurement
    processed_df = processed_df[processed_df['Mätning Spårvidd'].notna()].copy()
    
    # Position extraction using .loc
    processed_df.loc[:, 'Start_m'] = processed_df['Startläge'].apply(extract_position)
    processed_df.loc[:, 'End_m'] = processed_df['Slutläge'].apply(extract_position)
    
    # Calculate length
    processed_df.loc[:, 'length'] = abs(processed_df['End_m'] - processed_df['Start_m'])
    
    # Extract measurement date
    processed_df.loc[:, 'Measurement Date'] = processed_df['Mätning Spårvidd'].apply(extract_measurement_date)
    
    # Drop intermediate position columns
    processed_df.drop(['Start_m', 'End_m'], axis=1, inplace=True)
    
    return processed_df

# Process the combined DataFrame
processed_df = process_dataframe(combined_df)

# Remove columns with all empty values
processed_df = processed_df.dropna(axis=1, how='all')

We remove the following columns (no additional info).
- Event Category
- Event Type
- Räl Beskrivning Yttersträng 
- Räl Beskrivning Innersträng
- _Skapat_av
- Skriptnamn
- Mätvagn

In [4]:
# Columns to remove
columns_to_remove = [
    'Event Category', 'Event Type', 'Räl Beskrivning Yttersträng', 
    'Räl Beskrivning Innersträng', '_Skapat_av', 'Skriptnamn', 'Mätvagn'
]

# Drop the specified columns if they exist in the DataFrame
processed_df= processed_df.drop(columns=[col for col in columns_to_remove], errors='ignore')

We can now export the combined cleaned dataframe to one excel file with different thematic sheets for further visualisation in Power BI.

In [5]:
# Step 6: Export the combined DataFrame to a single Excel sheet
output_file = 'combined_data.xlsx'
processed_df.to_excel(output_file, sheet_name='All Data', index=False)